In [1]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip -q install transformers datasets emoji tldextract wordfreq openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00


In [3]:
import re
import os
import math
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import tldextract
import emoji
from wordfreq import zipf_frequency

# =========================
# CONFIG (change if needed)
# =========================
INPUT_XLSX = "/content/linkedinFauxPostDetection_Dataset.xlsx"  # <-- put your path here (upload or from Drive)
TEXT_COL   = "Data"
LABEL_COL  = "Label (Fake / Real )"

LABEL_MAP = {"fake": 1, "real": 0}  # Fake=1, Real=0
TEST_SIZE = 0.10
VAL_SIZE  = 0.10
RANDOM_STATE = 42

BERT_MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 256  # 128/256/512; 256 is a good balance

OUTPUT_DIR = "/content/lfpd_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Config OK ✅")


Config OK ✅


In [4]:
# Load
df_raw = pd.read_excel(INPUT_XLSX)

# Drop unnamed empty columns
df = df_raw.loc[:, ~df_raw.columns.astype(str).str.startswith("Unnamed")].copy()

# Keep only needed columns
assert TEXT_COL in df.columns and LABEL_COL in df.columns, "Check TEXT_COL and LABEL_COL names."
df = df[[TEXT_COL, LABEL_COL]].copy()

# Drop empty
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()

df = df[(df[TEXT_COL] != "") & (df[LABEL_COL] != "")]
df = df.drop_duplicates(subset=[TEXT_COL]).reset_index(drop=True)

# Normalize label
df[LABEL_COL] = df[LABEL_COL].str.lower().str.replace(r"[^a-z]", "", regex=True)
df["label"] = df[LABEL_COL].map(LABEL_MAP)

missing = df["label"].isna().sum()
if missing > 0:
    print(f"Warning: {missing} rows with unknown labels dropped.")
    df = df[~df["label"].isna()]

df["label"] = df["label"].astype(int)
df = df.rename(columns={TEXT_COL: "text"})
df = df[["text", "label"]].reset_index(drop=True)

print(df.head(3))
print("Class balance:", df["label"].value_counts(normalize=True))
print("Rows after cleaning:", len(df))


                                                text  label
0  🚀 Exciting Internship Opportunity at GAO Tek I...      1
1  Urgent hiring! Pay starts at $8,000/week. Send...      1
2  Exciting opportunity to earn $10,000/month. No...      1
Class balance: label
1    0.624655
0    0.375345
Name: proportion, dtype: float64
Rows after cleaning: 2539


In [5]:
URL_RE = re.compile(r"https?://[^\s)]+", flags=re.I)
GOOGLE_FORM_RE = re.compile(r"https?://(forms\.gle|docs\.google\.com/forms)[^\s)]+", re.I)
SHORTENER_RE = re.compile(r"https?://(bit\.ly|tinyurl\.com|cutt\.ly|is\.gd|rb\.gy|t\.co|rebrand\.ly|linktr\.ee)/[^\s)]+", re.I)
EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@([A-Za-z0-9.-]+\.[A-Za-z]{2,})")
PHONE_RE_IN = re.compile(r"(\+?91[\-\s]?)?[6-9]\d{9}")

HASHTAG_RE = re.compile(r"#\w+")
PUNCT_RUN_RE = re.compile(r"([!?])\1{2,}")  # 3+ ! or ?

CURRENCY_RE = re.compile(r"[₹$€]")
MONEY_PER_PERIOD_RE = re.compile(r"([₹$€]\s?\d[\d,]*(\.\d+)?\s?(\/|per\s)(day|week|month|hour))", re.I)

# Keywords (customize easily)
URGENCY_WORDS = [
    "urgent", "apply now", "hurry", "last chance", "closing soon", "limited slots", "immediately", "asap"
]
UPFRONT_FEE_WORDS = [
    "registration fee", "security deposit", "training fee", "pay to apply", "processing fee"
]
PII_WORDS = ["aadhaar", "aadhar", "pan", "dob", "otp", "bank details", "credit card", "debit card", "upi"]

PLATFORM_BYPASS_CUES = ["dm me", "whatsapp", "wa.me", "t.me", "telegram", "fill this form", "google form", "apply via form"]

FREE_EMAIL_DOMAINS = {"gmail.com", "yahoo.com", "outlook.com", "hotmail.com", "rediffmail.com", "yopmail.com"}

SUSPICIOUS_TLDS = {"xyz", "top", "live", "icu", "click"}


In [6]:
def count_emojis(s: str) -> int:
    return len(emoji.emoji_list(s))

def caps_ratio(s: str) -> float:
    letters = [ch for ch in s if ch.isalpha()]
    if not letters:
        return 0.0
    caps = sum(1 for ch in letters if ch.isupper())
    return caps / len(letters)

def max_punct_run(s: str) -> int:
    runs = PUNCT_RUN_RE.findall(s)
    # findall returns list of tuples due to capturing; we can compute via lengths directly:
    max_run = 0
    current = 1
    prev = ""
    for ch in s:
        if ch in "!?" and ch == prev:
            current += 1
        else:
            max_run = max(max_run, current)
            current = 1
        prev = ch
    max_run = max(max_run, current)
    return max_run if max_run >= 3 else 0

def oov_ratio(s: str, zipf_threshold: float = 2.5) -> float:
    # split to words (keep apostrophes inside words)
    tokens = re.findall(r"[A-Za-z][A-Za-z']+", s.lower())
    if not tokens:
        return 0.0
    oov = 0
    for tok in tokens:
        z = zipf_frequency(tok, "en")
        if z < zipf_threshold:
            oov += 1
    return oov / len(tokens)

def digit_density(s: str) -> float:
    if not s:
        return 0.0
    digits = sum(ch.isdigit() for ch in s)
    return digits / len(s)

def extract_domains(urls):
    domains = []
    tlds = []
    for u in urls:
        ext = tldextract.extract(u)
        domain = f"{ext.domain}.{ext.suffix}" if ext.suffix else ext.domain
        domains.append(domain.lower())
        tlds.append(ext.suffix.lower() if ext.suffix else "")
    return domains, tlds

def personal_email_domain_flag(emails):
    for e, dom in emails:
        if dom.lower() in FREE_EMAIL_DOMAINS:
            return 1
    return 0

def has_uncommon_tld(tlds):
    return int(any(t in SUSPICIOUS_TLDS for t in tlds if t))

def any_phrase_present(text, phrases):
    t = text.lower()
    return int(any(p in t for p in phrases))

def money_claims_count(text):
    return len(MONEY_PER_PERIOD_RE.findall(text))

def currency_symbol_count(text):
    return len(CURRENCY_RE.findall(text))

def url_char_ratio(text, urls):
    if not text:
        return 0.0
    total = sum(len(u) for u in urls)
    return total / len(text)

def extract_features(text: str) -> dict:
    urls = URL_RE.findall(text)
    emails = [(m.group(0), m.group(1)) for m in EMAIL_RE.finditer(text)]
    phones = PHONE_RE_IN.findall(text)
    hashtags = HASHTAG_RE.findall(text)

    domains, tlds = extract_domains(urls)

    feats = {
        "has_url": int(len(urls) > 0),
        "num_urls": len(urls),
        "has_google_form": int(GOOGLE_FORM_RE.search(text) is not None),
        "num_google_form": len(GOOGLE_FORM_RE.findall(text)),
        "has_shortener": int(SHORTENER_RE.search(text) is not None),
        "num_shortener": len(SHORTENER_RE.findall(text)),

        "has_email": int(len(emails) > 0),
        "num_emails": len(emails),
        "has_phone": int(len(phones) > 0),
        "num_phones": len(phones),

        "personal_email_domain_flag": personal_email_domain_flag(emails),
        "uncommon_tld_flag": has_uncommon_tld(tlds),

        "hashtag_count": len(hashtags),
        "emoji_count": count_emojis(text),
        "caps_ratio": caps_ratio(text),
        "punct_run_len_max": max_punct_run(text),
        "oov_ratio": oov_ratio(text, zipf_threshold=2.5),

        "digit_density": digit_density(text),
        "currency_symbol_count": currency_symbol_count(text),
        "money_claims_count": money_claims_count(text),
        "currency_per_period_flag": int(money_claims_count(text) > 0),

        "urgency_score": sum(1 for w in URGENCY_WORDS if w in text.lower()),
        "upfront_fee_flag": any_phrase_present(text, UPFRONT_FEE_WORDS),
        "pii_request_score": sum(1 for w in PII_WORDS if w in text.lower()),

        "platform_bypass_flag": any_phrase_present(text, PLATFORM_BYPASS_CUES),

        "url_char_ratio": url_char_ratio(text, urls),

        # Keep first domain & tld (textual, for potential later encoding)
        "primary_domain": domains[0] if domains else "",
        "primary_tld": tlds[0] if tlds else "",
    }
    return feats

# Apply feature extraction
features = df["text"].apply(extract_features).apply(pd.Series)
df_features = pd.concat([df, features], axis=1)

print("Feature columns:", [c for c in df_features.columns if c not in ("text", "label")][:15], "...")
print(df_features.head(2))



Feature columns: ['has_url', 'num_urls', 'has_google_form', 'num_google_form', 'has_shortener', 'num_shortener', 'has_email', 'num_emails', 'has_phone', 'num_phones', 'personal_email_domain_flag', 'uncommon_tld_flag', 'hashtag_count', 'emoji_count', 'caps_ratio'] ...
                                                text  label  has_url  \
0  🚀 Exciting Internship Opportunity at GAO Tek I...      1        0   
1  Urgent hiring! Pay starts at $8,000/week. Send...      1        0   

   num_urls  has_google_form  num_google_form  has_shortener  num_shortener  \
0         0                0                0              0              0   
1         0                0                0              0              0   

   has_email  num_emails  ...  currency_symbol_count  money_claims_count  \
0          1           1  ...                      0                   0   
1          0           0  ...                      1                   1   

   currency_per_period_flag  urgency_score  upfr

In [7]:
# First split train+val vs test
df_trainval, df_test = train_test_split(
    df_features, test_size=TEST_SIZE, stratify=df_features["label"], random_state=RANDOM_STATE
)

# Now split train vs val from trainval
val_frac_of_trainval = VAL_SIZE / (1 - TEST_SIZE)
df_train, df_val = train_test_split(
    df_trainval, test_size=val_frac_of_trainval, stratify=df_trainval["label"], random_state=RANDOM_STATE
)

for name, part in [("Train", df_train), ("Val", df_val), ("Test", df_test)]:
    print(name, "size:", len(part), "| class dist:", part["label"].value_counts(normalize=True).to_dict())

# Save CSVs (text + engineered features + label)
train_csv = os.path.join(OUTPUT_DIR, "train.csv")
val_csv   = os.path.join(OUTPUT_DIR, "val.csv")
test_csv  = os.path.join(OUTPUT_DIR, "test.csv")

df_train.to_csv(train_csv, index=False)
df_val.to_csv(val_csv, index=False)
df_test.to_csv(test_csv, index=False)

print("Saved:", train_csv, val_csv, test_csv)


Train size: 2031 | class dist: {1: 0.6243229935992122, 0: 0.37567700640078777}
Val size: 254 | class dist: {1: 0.6259842519685039, 0: 0.37401574803149606}
Test size: 254 | class dist: {1: 0.6259842519685039, 0: 0.37401574803149606}
Saved: /content/lfpd_outputs/train.csv /content/lfpd_outputs/val.csv /content/lfpd_outputs/test.csv


In [8]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

def tokenize_batch(texts, max_length=MAX_LENGTH):
    enc = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_attention_mask=True
    )
    return {
        "input_ids": np.array(enc["input_ids"], dtype=np.int32),
        "attention_mask": np.array(enc["attention_mask"], dtype=np.int32),
    }

def pack_npz(df_part, fname_npz):
    # Select numeric feature columns (exclude raw text, labels, and non-numeric)
    exclude = {"text", "label", "primary_domain", "primary_tld"}
    numeric_cols = [c for c in df_part.columns if c not in exclude and pd.api.types.is_numeric_dtype(df_part[c])]
    feats = df_part[numeric_cols].fillna(0.0).astype(float).values

    toks = tokenize_batch(df_part["text"].tolist(), max_length=MAX_LENGTH)
    labels = df_part["label"].values.astype(np.int32)

    np.savez_compressed(
        fname_npz,
        input_ids=toks["input_ids"],
        attention_mask=toks["attention_mask"],
        features=feats,
        labels=labels,
        feature_names=np.array(numeric_cols)
    )
    return numeric_cols

train_npz = os.path.join(OUTPUT_DIR, "bert_tokenized_train.npz")
val_npz   = os.path.join(OUTPUT_DIR, "bert_tokenized_val.npz")
test_npz  = os.path.join(OUTPUT_DIR, "bert_tokenized_test.npz")

feat_cols_train = pack_npz(df_train, train_npz)
feat_cols_val   = pack_npz(df_val,   val_npz)
feat_cols_test  = pack_npz(df_test,  test_npz)

print("Saved NPZs:", train_npz, val_npz, test_npz)
print("Numeric feature columns used:", len(feat_cols_train))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Saved NPZs: /content/lfpd_outputs/bert_tokenized_train.npz /content/lfpd_outputs/bert_tokenized_val.npz /content/lfpd_outputs/bert_tokenized_test.npz
Numeric feature columns used: 26


In [9]:
data = np.load(os.path.join(OUTPUT_DIR, "bert_tokenized_train.npz"), allow_pickle=True)
print("NPZ keys:", list(data.keys()))
print("Shapes:",
      data["input_ids"].shape,
      data["attention_mask"].shape,
      data["features"].shape,
      data["labels"].shape)
print("First 5 feature names:", data["feature_names"][:5])


NPZ keys: ['input_ids', 'attention_mask', 'features', 'labels', 'feature_names']
Shapes: (2031, 256) (2031, 256) (2031, 26) (2031,)
First 5 feature names: ['has_url' 'num_urls' 'has_google_form' 'num_google_form' 'has_shortener']


Preprocessing done

modelling start


In [10]:
!pip -q install transformers scikit-learn torchmetrics

import os, json, math, random, time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, AutoConfig
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from torch.optim import AdamW
from torchmetrics.functional import accuracy

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

OUTPUT_DIR = "/content/lfpd_outputs"
TRAIN_NPZ = os.path.join(OUTPUT_DIR, "bert_tokenized_train.npz")
VAL_NPZ   = os.path.join(OUTPUT_DIR, "bert_tokenized_val.npz")
TEST_NPZ  = os.path.join(OUTPUT_DIR, "bert_tokenized_test.npz")

BERT_MODEL_NAME = "bert-base-uncased"
NUM_CLASSES = 2  # Real(0), Fake(1)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

print("Paths OK")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 30.5 MB/s eta 0:00:00
Device: cpu
Paths OK


In [11]:
def load_npz(path):
    z = np.load(path, allow_pickle=True)
    return {
        "input_ids": z["input_ids"],
        "attention_mask": z["attention_mask"],
        "features": z["features"].astype(np.float32),
        "labels": z["labels"],
        "feature_names": [str(x) for x in z["feature_names"]],
    }

train = load_npz(TRAIN_NPZ)
val   = load_npz(VAL_NPZ)
test  = load_npz(TEST_NPZ)

print("Train shapes:",
      train["input_ids"].shape, train["attention_mask"].shape,
      train["features"].shape, train["labels"].shape)

# Standardize features (fit on train, apply to all)
scaler = StandardScaler()
train_feats_std = scaler.fit_transform(train["features"])
val_feats_std   = scaler.transform(val["features"])
test_feats_std  = scaler.transform(test["features"])

# For later inference, you may want to persist scaler params
scaler_params = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist(),
    "feature_names": train["feature_names"],
}
with open(os.path.join(OUTPUT_DIR, "feature_scaler.json"), "w") as f:
    json.dump(scaler_params, f)

print("Standardization done. Feature count:", train_feats_std.shape[1])


Train shapes: (2031, 256) (2031, 256) (2031, 26) (2031,)
Standardization done. Feature count: 26


In [12]:
class LFPDTokenizedDataset(Dataset):
    def __init__(self, ids, mask, feats, labels):
        self.input_ids = torch.tensor(ids, dtype=torch.long)
        self.attention_mask = torch.tensor(mask, dtype=torch.long)
        self.features = torch.tensor(feats, dtype=torch.float)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return self.labels.size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "features": self.features[idx],
            "labels": self.labels[idx],
        }

train_ds = LFPDTokenizedDataset(train["input_ids"], train["attention_mask"], train_feats_std, train["labels"])
val_ds   = LFPDTokenizedDataset(val["input_ids"],   val["attention_mask"],   val_feats_std,   val["labels"])
test_ds  = LFPDTokenizedDataset(test["input_ids"],  test["attention_mask"],  test_feats_std,  test["labels"])

BATCH_SIZE = 16
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=(DEVICE=="cuda"))
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=(DEVICE=="cuda"))
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=(DEVICE=="cuda"))
print("DataLoaders ready.")


DataLoaders ready.


In [13]:
class BertFeatsClassifier(nn.Module):
    def __init__(self, bert_name: str, feature_dim: int, num_classes: int = 2,
                 feat_hidden: int = 64, dropout_p: float = 0.2, freeze_bert: bool = False):
        super().__init__()
        self.config = AutoConfig.from_pretrained(bert_name)
        self.bert = BertModel.from_pretrained(bert_name)

        # Optional: freeze BERT encoder for speed (you can set False to fine-tune fully)
        if freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

        bert_out_dim = self.config.hidden_size  # 768 for bert-base
        self.text_dropout = nn.Dropout(dropout_p)

        # Small MLP over side features
        self.feat_net = nn.Sequential(
            nn.Linear(feature_dim, feat_hidden),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )

        self.classifier = nn.Sequential(
            nn.Linear(bert_out_dim + feat_hidden, 128),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(128, num_classes),
        )

    def forward(self, input_ids, attention_mask, features):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        # Use pooled_output (CLS) — already passed through a tanh dense layer inside BERT
        pooled = self.text_dropout(out.pooler_output)  # [B, 768]

        feat_repr = self.feat_net(features)            # [B, feat_hidden]
        concat = torch.cat([pooled, feat_repr], dim=1) # [B, 768+feat_hidden]
        logits = self.classifier(concat)
        return logits


In [14]:
# Compute class weights from training labels (Real=0, Fake=1)
class_counts = np.bincount(train["labels"], minlength=NUM_CLASSES)
class_weights = (class_counts.sum() / (NUM_CLASSES * class_counts + 1e-8)).astype(np.float32)
print("Class counts:", class_counts, " -> weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float, device=DEVICE))

def evaluate(model, data_loader):
    model.eval()
    all_logits, all_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            features = batch["features"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)

            logits = model(input_ids, attention_mask, features)
            all_logits.append(logits.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    preds = all_logits.argmax(axis=1)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, preds, labels=[0,1], average=None, zero_division=0)
    macro_f1 = f1.mean()
    acc = (preds == all_labels).mean()

    report = {
        "acc": acc,
        "macro_f1": macro_f1,
        "per_class": {
            "real": {"precision": float(p[0]), "recall": float(r[0]), "f1": float(f1[0])},
            "fake": {"precision": float(p[1]), "recall": float(r[1]), "f1": float(f1[1])},
        }
    }
    return report, preds, all_labels, all_logits

def pretty_report(tag, rep):
    print(f"\n[{tag}] acc={rep['acc']:.4f} | macro_f1={rep['macro_f1']:.4f} | "
          f"Fake-F1={rep['per_class']['fake']['f1']:.4f} (P={rep['per_class']['fake']['precision']:.4f}, "
          f"R={rep['per_class']['fake']['recall']:.4f})")



Class counts: [ 763 1268]  -> weights: [1.3309306 0.8008675]


In [ ]:
feature_dim = train_feats_std.shape[1]
model = BertFeatsClassifier(
    bert_name=BERT_MODEL_NAME,
    feature_dim=feature_dim,
    num_classes=NUM_CLASSES,
    feat_hidden=64,
    dropout_p=0.2,
    freeze_bert=False  # set True to train faster (but lower accuracy)
).to(DEVICE)

EPOCHS = 5
LR = 2e-5
WD = 0.01
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

best_val_fake_f1 = -1.0
patience = 2
no_improve = 0
best_path = os.path.join(OUTPUT_DIR, "best_model.pt")

for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        features = batch["features"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        logits = model(input_ids, attention_mask, features)
        loss = criterion(logits, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    train_rep, _, _, _ = evaluate(model, train_loader)
    val_rep, _, _, _ = evaluate(model, val_loader)
    pretty_report(f"Epoch {epoch} / Train", train_rep)
    pretty_report(f"Epoch {epoch} / Val", val_rep)

    val_fake_f1 = val_rep["per_class"]["fake"]["f1"]

    # Early stopping on Val Fake-F1
    if val_fake_f1 > best_val_fake_f1:
        best_val_fake_f1 = val_fake_f1
        no_improve = 0
        torch.save(model.state_dict(), best_path)
        print(f"✅ Saved new best model (Val Fake-F1={best_val_fake_f1:.4f}) -> {best_path}")
    else:
        no_improve += 1
        print(f"⚠️ No improvement [{no_improve}/{patience}]")
        if no_improve >= patience:
            print("⏹️ Early stopping.")
            break

print("Training done.")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Load best checkpoint
best_model = BertFeatsClassifier(
    bert_name=BERT_MODEL_NAME,
    feature_dim=feature_dim,
    num_classes=NUM_CLASSES,
    feat_hidden=64,
    dropout_p=0.2,
    freeze_bert=False
).to(DEVICE)

best_model.load_state_dict(torch.load(best_path, map_location=DEVICE))

test_rep, test_pred, test_true, test_logits = evaluate(best_model, test_loader)
pretty_report("TEST", test_rep)

print("\nClassification report (per class):")
print(classification_report(test_true, test_pred, target_names=["Real(0)","Fake(1)"], digits=4))

print("Confusion matrix [[TN, FP],[FN, TP]]:")
print(confusion_matrix(test_true, test_pred, labels=[0,1]))


In [ ]:
# Compute slice metrics using feature flags in VAL set (you can switch to TEST similarly)
# We'll re-open val NPZ to access feature_names & values
val_z = np.load(VAL_NPZ, allow_pickle=True)
feature_names = [str(x) for x in val_z["feature_names"]]
val_feats = val_feats_std  # standardized features we used

def idx_of(name):
    return feature_names.index(name) if name in feature_names else None

flag_names = ["has_google_form", "has_shortener", "money_claims_count", "platform_bypass_flag"]
flag_idx = {fn: idx_of(fn) for fn in flag_names}
flag_idx

# Make predictions on VAL to compute slice metrics
best_model.eval()
all_preds, all_labels, all_flags = [], [], {}

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        features = batch["features"].to(DEVICE)
        labels = batch["labels"]

        logits = best_model(input_ids, attention_mask, features)
        preds = logits.argmax(dim=1).cpu()

        all_preds.append(preds.numpy())
        all_labels.append(labels.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Reconstruct flags aligned to val order
val_flags = {fn: (val_feats[:, flag_idx[fn]] if flag_idx[fn] is not None else None) for fn in flag_names}

def slice_report(name, mask):
    y_true = all_labels[mask]
    y_pred = all_preds[mask]
    if len(y_true) == 0:
        print(f"[Slice: {name}] No samples.")
        return
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0,1], zero_division=0)
    print(f"[Slice: {name}] size={len(y_true)} | Fake-F1={f1[1]:.4f} (P={p[1]:.4f}, R={r[1]:.4f}) | Acc={(y_pred==y_true).mean():.4f}")

for name in flag_names:
    idx = flag_idx[name]
    if idx is None:
        print(f"[Slice: {name}] Not in features.")
        continue
    # For binary flags: >0 considered True; for count features: >0 means present
    mask = val_feats[:, idx] > 0
    slice_report(name, mask)


Inference (single post → prediction)

raw post text input lo

same feature extractor use karo

scaler apply + BERT tokenize

best model load karke label + probs + top signals return


In [ ]:
import json, numpy as np, torch
from transformers import AutoTokenizer
import torch.nn.functional as F

# Reuse: BertFeatsClassifier class, extract_features(), tokenizer config, feature_names from NPZ/scaler json
device = DEVICE if 'DEVICE' in globals() else ("cuda" if torch.cuda.is_available() else "cpu")

# Load artifacts
with open(f"{OUTPUT_DIR}/feature_scaler.json", "r") as f:
    scaler_params = json.load(f)
feature_names_serving = scaler_params["feature_names"]
scaler_mean = np.array(scaler_params["mean"], dtype=np.float32)
scaler_scale = np.array(scaler_params["scale"], dtype=np.float32)

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
feature_dim = len(feature_names_serving)

infer_model = BertFeatsClassifier(
    bert_name=BERT_MODEL_NAME, feature_dim=feature_dim,
    num_classes=2, feat_hidden=64, dropout_p=0.2, freeze_bert=False
).to(device)
infer_model.load_state_dict(torch.load(f"{OUTPUT_DIR}/best_model.pt", map_location=device))
infer_model.eval()

def standardize_features(vec_np: np.ndarray) -> np.ndarray:
    # vec_np shape: (feature_dim,)
    return (vec_np - scaler_mean) / (scaler_scale + 1e-8)

def vectorize_features(feat_dict: dict) -> np.ndarray:
    # Only numeric features that were used during training
    vals = []
    for name in feature_names_serving:
        v = feat_dict.get(name, 0.0)
        vals.append(float(v))
    return np.array(vals, dtype=np.float32)

def tokenize_single(text: str, max_length=256):
    enc = tokenizer(
        [text], padding="max_length", truncation=True, max_length=max_length, return_attention_mask=True
    )
    input_ids = torch.tensor(enc["input_ids"], dtype=torch.long)
    attention_mask = torch.tensor(enc["attention_mask"], dtype=torch.long)
    return input_ids, attention_mask

def top_signals(feats: dict, k=3):
    # Heuristic picks: choose binary/count features that are 1 or >0 and most indicative
    priority = [
        "has_google_form", "money_claims_count", "platform_bypass_flag",
        "has_shortener", "currency_per_period_flag", "personal_email_domain_flag",
        "upfront_fee_flag", "pii_request_score", "uncommon_tld_flag", "has_email", "has_phone"
    ]
    present = []
    for p in priority:
        if p in feats:
            try:
                val = float(feats[p])
                if val > 0:
                    present.append(p)
            except:
                pass
    return present[:k]

def predict_post(text: str, threshold: float = 0.50):
    # 1) engineered features
    feats = extract_features(text)
    vec = vectorize_features(feats)
    vec_std = standardize_features(vec)

    # 2) tokenize
    input_ids, attention_mask = tokenize_single(text, max_length=MAX_LENGTH)

    # 3) to device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    feats_t = torch.tensor(vec_std, dtype=torch.float, device=device).unsqueeze(0)

    # 4) forward
    with torch.no_grad():
        logits = infer_model(input_ids, attention_mask, feats_t)
        probs = F.softmax(logits, dim=1).cpu().numpy()[0]
    prob_real, prob_fake = float(probs[0]), float(probs[1])
    label = "fake" if prob_fake >= threshold else "real"

    return {
        "label": label,
        "prob_fake": prob_fake,
        "prob_real": prob_real,
        "threshold_used": threshold,
        "top_signals": top_signals(feats, k=3)
    }

# Demo
demo_text = "Urgent hiring! Earn $8000/week. Fill this Google Form: https://forms.gle/xyz and WhatsApp at wa.me/911234567890"
print(predict_post(demo_text, threshold=0.50))


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Use validation logits by re-running evaluate() but grab logits for probability
infer_model.eval()
all_logits, all_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        feats = batch["features"].to(device)
        labels = batch["labels"].cpu().numpy()
        logits = infer_model(ids, mask, feats).cpu().numpy()

        all_logits.append(logits)
        all_labels.append(labels)

all_logits = np.concatenate(all_logits, axis=0)
all_labels = np.concatenate(all_labels, axis=0)
probs = np.exp(all_logits - all_logits.max(axis=1, keepdims=True))
probs = probs / probs.sum(axis=1, keepdims=True)
prob_fake = probs[:, 1]

def evaluate_threshold(t):
    preds = (prob_fake >= t).astype(int)  # 1 = Fake
    p, r, f1, _ = precision_recall_fscore_support(all_labels, preds, labels=[0,1], average=None, zero_division=0)
    return {
        "thr": t,
        "acc": (preds == all_labels).mean(),
        "macro_f1": float(f1.mean()),
        "fake_p": float(p[1]), "fake_r": float(r[1]), "fake_f1": float(f1[1]),
        "real_p": float(p[0]), "real_r": float(r[0]), "real_f1": float(f1[0]),
    }

candidates = np.linspace(0.30, 0.70, 21)
results = [evaluate_threshold(float(t)) for t in candidates]

# Strategy 1: pick max Fake-F1
best_by_fake_f1 = max(results, key=lambda x: x["fake_f1"])
print("Best by Fake-F1:", best_by_fake_f1)

# Strategy 2: enforce Fake precision >= 0.70 then maximize recall
feasible = [r for r in results if r["fake_p"] >= 0.70]
best_prec_constraint = max(feasible, key=lambda x: x["fake_r"]) if feasible else best_by_fake_f1
print("Best with Fake-P>=0.70 (max recall):", best_prec_constraint)

# (Optional) pretty table
pd.DataFrame(results).round(4).sort_values("fake_f1", ascending=False).head(10)


In [ ]:
import pandas as pd

# Rebuild test predictions with probs for export
infer_model.eval()
t_logits, t_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        feats = batch["features"].to(device)
        labels = batch["labels"].cpu().numpy()
        logits = infer_model(ids, mask, feats).cpu().numpy()
        t_logits.append(logits); t_labels.append(labels)

t_logits = np.concatenate(t_logits, axis=0)
t_labels = np.concatenate(t_labels, axis=0)
t_probs = np.exp(t_logits - t_logits.max(axis=1, keepdims=True))
t_probs = t_probs / t_probs.sum(axis=1, keepdims=True)
t_pred = t_probs.argmax(axis=1)

# Load raw test.csv to get original text and feature flags
test_csv = pd.read_csv(f"{OUTPUT_DIR}/test.csv")
assert len(test_csv) == len(t_labels), "Mismatch between test csv and loader order."

df_err = pd.DataFrame({
    "text": test_csv["text"],
    "true_label": t_labels,
    "pred_label": t_pred,
    "prob_real": t_probs[:,0],
    "prob_fake": t_probs[:,1],
    "has_google_form": test_csv.get("has_google_form", 0),
    "has_shortener": test_csv.get("has_shortener", 0),
    "money_claims_count": test_csv.get("money_claims_count", 0),
    "platform_bypass_flag": test_csv.get("platform_bypass_flag", 0),
    "personal_email_domain_flag": test_csv.get("personal_email_domain_flag", 0),
    "uncommon_tld_flag": test_csv.get("uncommon_tld_flag", 0),
    "upfront_fee_flag": test_csv.get("upfront_fee_flag", 0),
    "pii_request_score": test_csv.get("pii_request_score", 0),
})

df_fp = df_err[(df_err.true_label==0) & (df_err.pred_label==1)].copy()  # predicted Fake but Real
df_fn = df_err[(df_err.true_label==1) & (df_err.pred_label==0)].copy()  # predicted Real but Fake

fp_path = f"{OUTPUT_DIR}/errors_false_positives.csv"
fn_path = f"{OUTPUT_DIR}/errors_false_negatives.csv"
df_fp.to_csv(fp_path, index=False)
df_fn.to_csv(fn_path, index=False)

print("Saved:")
print("FP ->", fp_path, " | count:", len(df_fp))
print("FN ->", fn_path, " | count:", len(df_fn))
df_fp.head(2), df_fn.head(2)


In [ ]:
more_flags = [
    "personal_email_domain_flag", "uncommon_tld_flag", "upfront_fee_flag",
    "pii_request_score", "has_email", "has_phone"
]

# Reuse validation probs/preds from Cell J
val_flags_full = {fn: (val_feats[:, feature_names.index(fn)] if fn in feature_names else None)
                  for fn in more_flags}

def slice_eval_by_flag(flag_name, greater_than_zero=True):
    idx = feature_names.index(flag_name) if flag_name in feature_names else None
    if idx is None:
        print(f"[Slice: {flag_name}] Not found in feature_names.")
        return
    mask = (val_feats[:, idx] > 0) if greater_than_zero else (val_feats[:, idx] == 0)
    y_true = all_labels[mask]
    y_pred = (prob_fake[mask] >= 0.5).astype(int)  # default threshold here; swap to tuned if you want
    if len(y_true) == 0:
        print(f"[Slice: {flag_name}] No samples.")
        return
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=[0,1], average=None, zero_division=0)
    print(f"[Slice: {flag_name}] size={len(y_true)} | Fake-F1={f1[1]:.4f} (P={p[1]:.4f}, R={r[1]:.4f}) | Acc={(y_pred==y_true).mean():.4f}")

for name in more_flags:
    slice_eval_by_flag(name, greater_than_zero=True)


In [ ]:
import shutil, datetime, json

version = datetime.datetime.now().strftime("v%Y%m%d_%H%M")
bundle_dir = f"{OUTPUT_DIR}/bundle_{version}"
os.makedirs(bundle_dir, exist_ok=True)

# Copy model + scaler
shutil.copyfile(f"{OUTPUT_DIR}/best_model.pt", f"{bundle_dir}/best_model.pt")
shutil.copyfile(f"{OUTPUT_DIR}/feature_scaler.json", f"{bundle_dir}/feature_scaler.json")

# Save a small config
cfg = {
    "bert_model": BERT_MODEL_NAME,
    "max_length": int(MAX_LENGTH),
    "feature_names": feature_names_serving,
    "exported_at": version
}
with open(f"{bundle_dir}/config.json", "w") as f:
    json.dump(cfg, f, indent=2)

print("Exported bundle ->", bundle_dir)


In [ ]:
!pip -q install fastapi uvicorn

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

# Load from bundle (update to your bundle path)
BUNDLE = bundle_dir  # from previous cell

# Artifacts
with open(f"{BUNDLE}/config.json", "r") as f:
    srv_cfg = json.load(f)
srv_tokenizer = AutoTokenizer.from_pretrained(srv_cfg["bert_model"])

srv_feature_names = srv_cfg["feature_names"]
with open(f"{BUNDLE}/feature_scaler.json", "r") as f:
    srv_scaler_params = json.load(f)
srv_mean = np.array(srv_scaler_params["mean"], dtype=np.float32)
srv_scale = np.array(srv_scaler_params["scale"], dtype=np.float32)

srv_model = BertFeatsClassifier(
    bert_name=srv_cfg["bert_model"], feature_dim=len(srv_feature_names),
    num_classes=2, feat_hidden=64, dropout_p=0.2, freeze_bert=False
).to(device)
srv_model.load_state_dict(torch.load(f"{BUNDLE}/best_model.pt", map_location=device))
srv_model.eval()

def srv_standardize(vec):
    return (vec - srv_mean) / (srv_scale + 1e-8)

def srv_vectorize(feat_dict: dict):
    return np.array([float(feat_dict.get(n, 0.0)) for n in srv_feature_names], dtype=np.float32)

def srv_tokenize(text: str, max_length: int):
    enc = srv_tokenizer([text], padding="max_length", truncation=True, max_length=max_length, return_attention_mask=True)
    ids = torch.tensor(enc["input_ids"], dtype=torch.long).to(device)
    mask = torch.tensor(enc["attention_mask"], dtype=torch.long).to(device)
    return ids, mask

app = FastAPI()

class PredictReq(BaseModel):
    text: str
    threshold: float = 0.50

@app.get("/health")
def health():
    return {"status": "ok", "model": srv_cfg["bert_model"], "max_length": srv_cfg["max_length"]}

@app.post("/predict")
def predict(req: PredictReq):
    feats = extract_features(req.text)
    vec = srv_vectorize(feats)
    vec_std = srv_standardize(vec)
    ids, mask = srv_tokenize(req.text, srv_cfg["max_length"])
    with torch.no_grad():
        logits = srv_model(ids, mask, torch.tensor(vec_std, dtype=torch.float, device=device).unsqueeze(0))
        pr = F.softmax(logits, dim=1).cpu().numpy()[0]
    prob_real, prob_fake = float(pr[0]), float(pr[1])
    label = "fake" if prob_fake >= req.threshold else "real"
    return {
        "label": label,
        "prob_fake": prob_fake,
        "prob_real": prob_real,
        "threshold_used": req.threshold,
        "top_signals": top_signals(feats, 3)
    }

# To run the server in Colab environment (for local testing):
# NOTE: In Colab, you'll need ngrok or gradio to expose externally; locally, just run uvicorn.
# uvicorn.run(app, host="0.0.0.0", port=8000)
print("FastAPI app ready. Run with: uvicorn.run(app, host='0.0.0.0', port=8000)")


In [ ]:
# Download cloudflared Linux binary & make it executable
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

# Sanity: show version (optional)
!./cloudflared --version


In [ ]:
import threading, uvicorn, time

# Start uvicorn in a background thread so the cell doesn't block
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to boot
time.sleep(2)
print("✅ Uvicorn started on http://0.0.0.0:8000")


In [ ]:
import subprocess, re, sys, time, threading

public_url = None

proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8000", "--no-autoupdate"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

def reader():
    global public_url
    for line in proc.stdout:
        sys.stdout.write(line)  # stream logs to cell output
        m = re.search(r"(https://[a-z0-9-]+\.trycloudflare\.com)", line)
        if m and public_url is None:
            public_url = m.group(1)

t = threading.Thread(target=reader, daemon=True)
t.start()

# wait up to 60s for URL
for _ in range(60):
    if public_url:
        break
    time.sleep(1)

print("\n🌍 Public URL:", public_url if public_url else "Not found yet")


In [ ]:
import requests, json
print("GET", public_url + "/health")
print(json.dumps(requests.get(public_url + "/health").json(), indent=2))
